In [2]:
# %matplotlib inline
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import lmfit as lm
import matplotlib.pyplot as plt
import matplotlib
from snmfem.models import edxs
import json
import snmfem.spectrum_fitting as sf
from snmfem.estimators import NMF

# Guide to this notebook

Every time the symbol ⚠️ appears, a user input is required in the cell below.

If a cell made a few windows pop up, please kill the windows once you're done. Otherwise every new plot will be displayed in top of the other (which may slow your computer down). 

Overview of the different steps :
- I. Load the data either a spectrum (go to step III.) or a spectrum image
- II. Select an area of interest in your spectrum image.
- III. Creates an energy scale (x) from your data for the fitting procedure.
- IV. Selects the regions of the spectrum were there are no peaks for background fitting.
- V. Creates the required objects (partial x and y) for the background fitting procedure.
- VI. Fitting the background. If this is not satisfactory go back to IV.

# I. Load data

Either a spectrum or a full spim

## Load spectrum
⚠️

In [2]:
spectrum = hs.load("46_ES-LP_L6_brg_test2.spx").isig[0.20 : 19.9]

⚠️ It is easier to fit the background on average spectra than on sum spectra. In pix_number, put the number of spectra you integrated.

In [3]:
pix_number = 43400
spectrum.data = spectrum.data /pix_number

## Load spectrum image

## .bcf file

⚠️

In [ ]:
bf,haadf,spim = hs.load("map.bcf")
spim.crop_signal1D(0.7,8.7)

## .emd file

⚠️ Be careful, depending on the number of elemental map or spectra you saved in your emd file you may need to inspect the output of ``hs.load``.
Simply put ``hs.load(file)`` in a new cell locate the haadf and the spim.

In [ ]:
s = hs.load("1537 IO-BTO 790 kx EDS-HAADF SI.emd")

In [ ]:
spim = s[-1].isig[0.6:]

# Bin the spectrum image

In [ ]:
def bin_spim(data,n,m):
    # return a matrix of shape (n,m)
    bs = data.shape[0]//n,data.shape[1]//m  # blocksize averaged over
    k = data.shape[2]
    return np.reshape(np.array([np.sum(data[k1*bs[0]:(k1+1)*bs[0],k2*bs[1]:(k2+1)*bs[1]],axis=(0,1)) for k1 in range(n) for k2 in range(m)]),(n,m,k))

n,m = 256,256
spim.data = bin_spim(spim.data,n,m)

spim.axes_manager[0].scale /= spim.axes_manager[0].size // n 
spim.axes_manager[1].scale /= spim.axes_manager[1].size // m 
spim.axes_manager[0].size = n
spim.axes_manager[1].size = m

In [ ]:
spim.save("1537_Robin_cropped.hspy")

# II. Select an area of interest

⚠️ 3 windows will pop up. 
- One window correspond to a 2D representation of the spim (sum along the energy direction) a green rectangle will be displayed on top of it, it corresponds to the selected area. 
- One window correspond to the first pixel (top left) of the spectrum image (you can ignore it). 
- The last window correspond to the average of the selected spectra

In [ ]:
    haadf_ROI=hs.roi.RectangularROI(10*spim.axes_manager[0].scale,
                                10*spim.axes_manager[1].scale,
                                40*spim.axes_manager[0].scale,
                                40*spim.axes_manager[1].scale)
    # arguments = left, top, right, bottom in pixels*scale
    spim.plot()
    spim_ROI=haadf_ROI.interactive(spim)
    # The HAADF and SPIM windows with their respective ROIs. The two ROIs are connected so that the selected area in the HAADF is the same as the one on the SPIM.
    
    spectrum=hs.interactive(spim_ROI.mean,
               event=spim_ROI.axes_manager.events.any_axis_changed,
               recompute_out_event=None)
    spectrum.plot(True)

# III. Calibrate x

In [4]:
offset = spectrum.axes_manager[0].offset
scale = spectrum.axes_manager[0].scale
size = spectrum.axes_manager[0].size

x = np.linspace(offset,size*scale+offset,num = size)

# IV. Select areas for bckgd fitting

⚠️ Input in the ``span_number`` the number of selection areas you wish to use. Once you execute the cell, a window will pop with a few green areas and a red spectrum. The green areas correspond to the regions were the background will be fitted. You can, drag, enlarge and reduce these green areas.

In [ ]:
span_number = 7

def selection_areas(number,spectrum,scale) :
    spectrum.plot()
    roi_list = []
    for i in range(number) :
        roi_list.append(hs.roi.SpanROI(i+200*scale,i+400*scale))
        roi_list[-1].interactive(spectrum)
    return roi_list

spans = selection_areas(span_number,spectrum,scale)

In [ ]:
list_energies = [[roi.left,roi.right] for roi in spans]

⚠️ Execute this cell only if you want to use a previous selection of areas for the background fitting.

In [5]:
list_energies = [[0.8014116348999999, 0.8514658053999999], [1.3820400126999997, 1.4020616808999997], [1.9526575563999997, 1.9826900586999998], [2.4231667590999995, 2.5332859341999994], [3.3241418280999997, 3.5243585101], [4.2651602335, 4.965918620499999], [6.6477387493, 6.8479554313], [11.8333508131, 14.9867635546]]

# VI. Fitting the continuum

In [12]:
part_x, part_y, sum_boola = sf.make_partial_xy(list_energies,spectrum,x)

example = {"b0" : 1.0e-1, "b1" : 1.0e-5, "params_dict" : {
    "Det" : "SDD_efficiency.txt", 'Abs' : {
        "thickness" : 100e-7, "toa" : 22, "density" : 4.5},
        "elements_dict" : {
            "Cu" : 0.1,
            "Fe" : 0.1,
            "Sm" : 0.1,
            "Nd" : 0.1,
            "Mg" : 0.1,
            "Si" : 0.1,
            "Pb" : 0.1,
            "Al" : 0.1,
            "Hf" : 0.1,
            "Ca" : 0.1
        }
    }
}

# {"detection" : {
#     "thickness" : 450e-4,
#     "elements_dict" : {
#         "Si" : 1.0
#     }
# },"layer" : {
#     "thickness" : 10e-7,
#     "elements_dict" : {
#         "Si" : 0.3,
#         "O" : 0.3,
#         "Al" : 0.3
#     }
# }}}

# example = {"b0" : 1.0, "b1" : 1.0,"params_dict" : {"Det" : "SDD_efficiency.txt","Abs" : {
# "thickness" : 300e-7
# }}, "elements_dict" : {
#     "Al" : 0.15,
#     "Zn" : 0.15,
#     "Mg" : 0.15,
#     "Cu" : 0.15,
#     "Zr" : 0.15
#     }}

pars = sf.ndict_to_params(example)

pars["params_dict__Abs__thickness"].vary = False
pars["params_dict__Abs__toa"].vary = False
pars["params_dict__Abs__density"].vary = False

out = lm.minimize(sf.residual, pars, args=(part_x,), kws={'data': part_y})
print(lm.fit_report(out))

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 51
    # data points      = 446
    # variables        = 2
    chi-square         = 9.6698e-04
    reduced chi-square = 2.1779e-06
    Akaike info crit   = -5812.57560
    Bayesian info crit = -5804.37496
##  Warning: uncertainties could not be estimated:
    b0:                              at boundary
[[Variables]]
    b0:                              5.8423e-09 (init = 0.1)
    b1:                              8.3292e-07 (init = 1e-05)
    params_dict__Abs__thickness:     1e-05 (fixed)
    params_dict__Abs__toa:           22 (fixed)
    params_dict__Abs__density:       4.5 (fixed)
    params_dict__elements_dict__Cu:  0.1 (fixed)
    params_dict__elements_dict__Fe:  0.1 (fixed)
    params_dict__elements_dict__Sm:  0.1 (fixed)
    params_dict__elements_dict__Nd:  0.1 (fixed)
    params_dict__elements_dict__Mg:  0.1 (fixed)
    params_dict__elements_dict__Si:  0.1 (fixed)
    params_dict__elements_dict__Pb:  0

### Plotting the results

The red curves corresponds to the background model, the black one to the data, and the grey area correspond to the selected green areas

In [ ]:
y = spectrum.data
bkgd = sf.residual(out.params,x)
plt.plot(x,y,"ko-",markersize=1.5,label="exp",markevery=10)
plt.fill_between(x,0,y,where=sum_boola,label="fit windows",color="0.8")
plt.xlabel("Energy (keV)",fontsize=22)
plt.xticks(fontsize=20)
plt.ylim(0,1)
plt.yticks(fontsize=20)
plt.ylabel("Intensity",fontsize=22)
plt.plot(x,bkgd,"r-",linewidth=1,label="fit")
plt.legend(fontsize=22)

In [ ]:
from snmfem.models.absorption_edxs import *

In [ ]:
A = absorption_correction(part_x,thickness = 3.2214e-10,toa = 90, density = None,elements_dict = {
    "Al" : 0.15,
    "Zn" : 0.15,
    "Mg" : 0.15,
    "Cu" : 0.15,
    "Zr" : 0.15
    })

D = det_efficiency_from_curve(part_x,"SDD_efficiency.txt")

brem = part_y /(A*D)

In [ ]:
plt.plot(1/part_x,brem,"ko",label = "corrected experimental data")
plt.plot(1/part_x,0.0215*1/part_x, label = "f(x) = 0.0215*1/x")
plt.xlabel("1/E",fontsize = 22)
plt.legend(fontsize = 22)

# VII. Select the interesting part of the spectrum

⚠️ A window will pop up with a red spectrum and a green area. With the green areas, you can select the energy range you want to quantify. Usually the low energy region (below 1keV) is not very well fitted by the background model. Hence you probably want to avoid the quantification of low energy characteristic X rays. 

If you feel brave enough you can uncomment some parts of the background modelling part and achieve a better background fit at low energies. At your own risks.

In [ ]:
s_bkgd = hs.signals.Signal1D(bkgd)
s_bkgd.axes_manager[0].offset = offset
s_bkgd.axes_manager[0].scale = scale

spectrum.plot()
ax = spectrum._plot.signal_plot.ax
ax.add_line(matplotlib.lines.Line2D(spectrum._plot.axis.axis,bkgd.data))
roi = hs.roi.SpanROI(200*scale,400*scale)
roi.interactive(spectrum)

### Accepting the choice of quantification energy range

In [ ]:
charac_xrays = (spectrum.isig[roi.left:roi.right].data - s_bkgd.isig[roi.left:roi.right].data).clip(min=0)
new_offset = spectrum.isig[roi.left:roi.right].axes_manager[0].offset
new_size = spectrum.isig[roi.left:roi.right].axes_manager[0].size

# VIII. Fitting the characteristic Xrays

### Initialisation of the characteristic Xray model
⚠️ Put the list of elements (by atomic numbers) you want to quantify in ``elt_list``. 

``G_calib`` is a small offset you may want to add so that the peaks are at the right energy. 

Note that depending on the energy range you chose, not all the elements you put are fitted (e.g. Oxygen will not be taken into account if your energy range starts at 1 keV)

In [ ]:
#elt_list = [29,30,31,40,78,81,12,13,14,20,26,92,71,72,60,62]
elt_list = ["Cu", "Fe", "Tl", "Mn", "Co", "Cr", "Ga", "La"]
G_calib = -0.005
density = 5.5
thickness = 300e-7
toa = 22

In [ ]:
db = "default_xrays.json"

true_elt_list = []
for elt in elt_list : 
    mod_t = edxs.EDXS(new_offset,new_size,scale,{},db_name = db,energy_range = [new_offset,new_size*scale+new_offset])
    mod_t.generate_g_matr(elements_list = [elt],norm=False)
    G_t = mod_t.G
    if G_t.shape[1] == 1 : 
        true_elt_list.append(elt)
    
mod = edxs.EDXS(new_offset + G_calib,new_size,scale,{},db_name = db,energy_range = [new_offset,new_size*scale+new_offset])
mod.generate_g_matr(elements_list = true_elt_list,norm=False, density = density, thickness = thickness, toa = 22)
G = mod.G
new_x = np.linspace(new_offset,new_size*scale+new_offset,num = new_size)
print("List of elements with Xray lines in the energy range :")
print(true_elt_list)

### Fitting the characteristic Xrays

In [ ]:
estimator = NMF(n_components= 1,max_iter=2000,force_simplex = True,tol = 1e-8)
estimator.fit_transform(G=G, A=np.array([1])[:,np.newaxis], X=charac_xrays[:,np.newaxis])
P = estimator.P_

⚠️ If you want to fix the values of P you obtained in a previous fit put them in ``fixed_elts`` and execute the two following cells. 

For a two steps quantification : Chose a first energy range. Perform a first fit, which initialise P with all the elements. Then go back to the energy range selection, ignore the cell above, chose the fixed elements and perform the fit with fixed elements below. 

In [ ]:
fixed_elts = [71,72]

In [ ]:
inds = [true_elt_list.index(elt) for elt in fix_elts]
trunc_elt_list = [v for i,v in enumerate(true_elt_list) if i not in frozenset(inds)] 
trunc_G = np.delete(G, inds, axis=1)
fixed_P = P[inds]
fixed_G = G[:,inds]
estimator = NMF(n_components= 1,max_iter=2000,force_simplex = True,tol = 1e-8)
estimator.fit_transform(G=trunc_G, A=np.array([1])[:,np.newaxis], X=charac_xrays[:,np.newaxis])
trunc_P = estimator.P_
P = np.concatenate((trunc_P,fixed_P))
G = np.hstack((trunc_G,fixed_G))
true_elt_list = trunc_elt_list + fixed_elts

# IX. Plot the results

⚠️ The popping window will display the bkgd substracted experimental spectrum, the fitted characteristic Xrays model in black and red. Every element composing the model will be displayed with an offset : ``components_offset``. You can change its value below. 

If you want to display the names of the lines of an element that was used for quantification execute the next cells below before kill the window.

In [ ]:
components_offset = -0.1

In [ ]:
linestyles = [":","--","-."]

plt.plot(new_x,charac_xrays,"ko-",label="exp",markevery = 10)
plt.plot(new_x,G@P,"r",linewidth = 3,label="theo")
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.xlabel("energy (keV)",fontsize = 18)
plt.ylabel("Intensity", fontsize = 18)

for i in range(G.shape[1]) :
    ls_string = linestyles[i%len(linestyles)] + "C{}".format(i%9)
    plt.plot(new_x,G[:,i]*P[i]+components_offset,ls_string,label=str(true_elt_list[i]),linewidth=3)


plt.legend(fontsize=18)
plt.tight_layout()

⚠️ You can change the `select_elt` to display the lines of this element. (It is probably going to messy as a lot of lines have very close energy). Execute this cell if you want to identify some lines.  

In [ ]:
select_elt = "72"

In [ ]:
with open("xray_lines.json","r") as f :
    lines_dict = json.load(f)["table"]    

for line in lines_dict[select_elt].keys() :
    if (lines_dict[select_elt][line]["energy"]> new_offset) and (lines_dict[select_elt][line]["energy"]<new_size*scale+new_offset) :
        plt.text(lines_dict[select_elt][line]["energy"],0.0,line,fontsize=18)


# X. Printing the concentrations

Full summary of normalized concentrations.

In [ ]:
r = P/P.sum(axis=0)
for i in range(P.shape[0]) :
    print("Z : ",true_elt_list[i],"  rel conc : ",r[i])

⚠️ You can remove some elements of the results, normalizing on the remaining elements.

In [ ]:
ignored_elts = [29,30]

In [ ]:
inds = [true_elt_list.index(elt) for elt in ignored_elts]
remain_elt_list = [v for i,v in enumerate(true_elt_list) if i not in frozenset(inds)] 
remain_P = np.delete(P, inds, axis=0)
r = remain_P / remain_P.sum(axis=0)
for i in range(r.shape[0]) :
    print("Z : ",remain_elt_list[i],"  rel conc : ",r[i])